In [ ]:
from glob import glob
import sys, time, os, asyncio
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
from scipy.signal import find_peaks
from scipy.interpolate import interp1d
from astropy.time import Time, TimeDelta
from scipy.interpolate import UnivariateSpline


from lsst_efd_client import EfdClient
from lsst.ts.idl.enums import MTM1M3

from lsst.summit.utils.efdUtils import calcNextDay, getEfdData

import lsst.sitcom.vandv.m1m3.sitcomtn81.sitcomtn_81_identify_oscillations as sitcomtn81

from tqdm import tqdm

import logging

summit_utils_logger = logging.getLogger("lsst.summit.utils")
summit_utils_logger.setLevel(logging.ERROR)

%matplotlib inline
%load_ext lab_black
%load_ext autoreload
%autoreload 2

# Characterizaiton of topple block 


Timestamps of the topple block slews:
Dayobs,  seqnum, eventnum, time (utc)
20231129, 73, 13, 1701269673.9407606
20231129, 90, 30, 1701298313.7859175
20231129, 97, 33, 1701299609.625537
20231129, 99, 38, 1701299684.265059
20231129, 100, 49, 1701299719.8852365
20231129, 100, 54, 1701299775.7053618
20231129, 107, 61, 1701302257.7229214
20231129, 114, 63, 1701303679.5524187
20231129, 120, 70, 1701305359.8571882
20231129, 365, 79, 1701316217.89929
20231129, 385, 80, 1701318835.4638815
20231129, 387, 81, 1701318835.4638815
20231129, 387, 82, 1701318878.6672893
20231129, 399, 84, 1701320922.0731387
20231129, 811, 101, 1701332665.7784734

In [ ]:
def add_timestamp(data):
    """
    Adds a correct timestamp column in UTC format to the provided data if
    not present.

    Parameters
    ----------
    data : DataFrame
        The data to which the timestamp will be added.

    Returns
    -------
    DataFrame
        The data with the added 'snd_timestamp_utc' column.
    """
    if "snd_timestamp_utc" not in data.columns:
        data["snd_timestamp_utc"] = Time(
            data["private_sndStamp"], format="unix_tai"
        ).unix
    return data


def get_data(events_frame, client, train=False):
    table_dict = {
        "m1m3_hp_actuator": "lsst.sal.MTM1M3.hardpointActuatorData",
    }
    query_dict = {}
    for key in table_dict.keys():
        query_dict[key] = []
    for j, time in tqdm(enumerate(events_frame["times"])):
        event_time = Time(events_frame["times"][j] - 2, format="unix")

        for key in table_dict.keys():
            hpcols = ["private_sndStamp"] + [f"measuredForce2"]  # for i in range(6)]
            query = getEfdData(
                client=client,
                topic=table_dict[key],
                columns=hpcols,
                begin=event_time,
                end=event_time,
                prePadding=60,
                postPadding=60,
            )
            query["event_num"] = j
            query["seqNum"] = events_frame["seq_num"][j]
            query["event_time"] = events_frame["times"][j] + 3
            query = add_timestamp(query)

            # query["block"] = block
            query["oscillation_bool"] = 0
            query["delta_time"] = (
                query["snd_timestamp_utc"] - events_frame["times"][j] + 3
            )
            if train:
                sel = query["delta_time"] >= -5
                sel &= query["delta_time"] < 1
                query.loc[sel, "oscillation_bool"] = 1

            query_dict[key].append(query)
    for key in table_dict.keys():
        query_dict[key] = pd.concat(query_dict[key])
    sorted_keys = sorted(query_dict.keys())
    initial_key = sorted_keys[0]
    merged_df = query_dict[initial_key].sort_index()
    # Define your tolerance for matching
    tolerance = 0.03  # pd.Timedelta('0.03 seconds')

    # Iterate over the remaining DataFrames and merge them
    for key in sorted_keys[1:]:
        merged_df = pd.merge_asof(
            merged_df,
            query_dict[key].sort_index(),
            left_on="snd_timestamp_utc",
            right_on="snd_timestamp_utc",
            tolerance=tolerance,
            direction="nearest",
            suffixes=("", "_" + key),
        )
    merged_df.reset_index(drop=True, inplace=True)

    return merged_df

In [ ]:
days_with_events = [
    i for i in days_with_events if (float(i) > 20231120) & (float(i) < 20231205)
]

In [ ]:
days_with_events

In [ ]:

day = 20231129
eventMaker = TMAEventMaker()
events = eventMaker.getEvents(int(day))

events_frame = pd.read_csv(f"./sitcomtn81_data/oscillation_events_20231129.csv")
merged_df = get_data(events_frame, client)

In [ ]:
good = [13, 30, 33, 38, 49, 54, 61, 63, 70, 79, 80, 81, 82, 84, 101]

In [ ]:
offset_list = [10, 2, 3.5, 2, 1, 1.5, 2, 4, 9, 4, 1, 1, 8.5, 7.5, -21]
offset_dict = {}
for l, k in enumerate(good):
    offset_dict[k] = offset_list[l]

In [ ]:
l = 1
plt.figure(dpi=175, figsize=(10, 8 * l))
j = 0
ptp_list = []
offset = 2000
for i in np.unique(merged_df["event_num"]):  # [k : k + 30 * l]:
    if i not in good:
        continue
    plt.title(f"{day} slews with oscillations")
    subframe = merged_df.copy()
    subframe = subframe[subframe["event_num"] == i]
    seq_val = np.unique(subframe["seqNum"])
    # block_num = np.unique(subframe["block"])
    if len(subframe) > 0:
        ptp_range = (subframe["delta_time"] - offset_dict[i] > 0) & (
            subframe["delta_time"] - offset_dict[i] < 4
        )

        plt.plot(
            subframe["delta_time"][ptp_range] - offset_dict[i],
            subframe["measuredForce2"][ptp_range]
            - subframe["measuredForce2"][subframe["delta_time"] < 0].mean()
            - offset * j,
            label=f"{seq_val[0]}",
        )
        ptp_range = (subframe["delta_time"] - offset_dict[i] > 0) & (
            subframe["delta_time"] - offset_dict[i] < 4
        )
        ptp_list.append(np.ptp(subframe["measuredForce2"][ptp_range]))
        j += 1
if l == 1:
    plt.legend(title="slew number")
if l == 4:
    plt.legend(fontsize=9.2, loc=7)
# plt.axvline(0, c="k", ls="dashed")
plt.ylabel(f"$\Delta$ HP 2 force [N] - {offset} N offset")
plt.xlabel("Time [s]")
plt.xlim(-10, 25)
for l in np.arange(0, 5, 1):
    plt.axvline(l, ls="dashed", c="k")
# plt.savefig(f"./sitcomtn81_data/plots/osc_hp2_{day}.png")

# plt.close()

In [ ]:
print(np.mean(ptp_list), np.std(ptp_list))

In [ ]:
seq_list = []
for row in good:
    day_obs, seq_num, times = events_frame.loc[row, ["day_obs", "seq_num", "times"]]
    print(f"{day_obs}, {seq_num}, {row}, {times}")
    seq_list.append(seq_num)
    events_frame.loc[row, ["day_obs", "seq_num", "times"]].to_csv(
        f"{datapath}/cleaned_events{day}.csv"
    )